# Script para web scraping Strava

>O script se baseia no uso de 2 bibliotecas, a ***Selenium*** e a ***Beatiful Soup***. Sendo a primeira responsável pela navegação nas páginas, isto é, desde login até movimentação da página, acessando hyperlinks e botões em java Script. Já a segunda é responsável pela cópia do código fonte, a partir da configuração de paramêtros, e retorno das chamadas. Em grandes linhas, buscamos a tabela onde os dados estão, e posteriormente extraimos a informação necessária.

*Necessário: Instalação das bibliotecas Selenium e BeautifulSoup, e também do webdriver do chrome para que o navegador seja acessado remotamente.

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

1. Inicialização do navegador remotamente, e carregamento da página inicial do strava. O tempo de 5s é para o carregamento inicial.
2. Solicitação de login e senha. 
3. O campo usuário e senha é localizado na página através da biblioteca Selenium, e posteriormente é preenchido e clicado.
4. Substituir o campo ***url_evento*** pela url do evento do strava, ou seja, a corrida.

## Alterar campos

In [ ]:
email=input(prompt="Digite seu email strava:")
senha=input(prompt="Digite sua senha strava:")

## Alterar campos

In [ ]:
driver = webdriver.Chrome()
driver.get('https://www.strava.com/login')
time.sleep(5)
url_evento = 'https://www.strava.com/segments/19286937'
export = "CHICAGO-001.csv"
ultima_pagina = 850


A identificação dos campos ***usuário*** e ***senha*** é feito através da funcionalidade "inspeção do código fonte".

In [ ]:
username = driver.find_element_by_id("email")
password = driver.find_element_by_id("password")


username.send_keys(email)
password.send_keys(senha)
driver.find_element_by_id("login-button").click() #encontrar o elemento botão
time.sleep(2)

driver.get(url_evento)
time.sleep(2)


> São utilizados 5 listas para o armazenamento das informações, sendo elas:
* results (n x 7) - ['classificacao','nome', 'ano','pace', 'fc', 'vam','tempo_liquido'] sem tratamento
* result_faixa_etaria (n x 1) - faixas etárias
* result_sexo (n x 1) - masculino / feminino
* data (n x 1) - ['classificacao','nome', 'ano','pace', 'fc', 'vam','tempo_liquido'] sem valores nulos
* link_atividades (n x 1) - link para ser direcionado aos detalhes do evento que o  atleta tenha participado

Foram criados ainda 3 dicionários, onde o índice é a posição para localização (xpath) do item:

Ex: ``element=driver.find_element_by_xpath('//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/ul/li['+str(se)+']/a')``

O valor ***se*** e ***fe*** são iterados no laço ***for*** para os indices do sexo e faixa etária.
Futuramente o dicionário de ***classe_peso*** poderá ser utilizado, nesse momento os dados não foram captados devido a uma independência dos filtros de **faixa_etaria e classe_peso**. Poderá ser implementando um script complementar para o preenchimento desse campo e **merge** com os demais campos.

In [ ]:
results = []
result_faixa_etaria = []
result_sexo=[]
data = []
link_atividades = []

faixa_etaria={'2':'19 e abaixo','3':'20 a 24','4':'25 a 34','5':'35 a 44','6':'45 a 54','7':'55 a 64','8':'65 a 69',
              '9':'70 a 74','10':'75+'}
sexo={'2':'Homens','3':'Mulheres'}
classe_peso={'54 kg e abaixo','55 a 64 kg', '75 a 84 kg','85 a 95 kg','95 kg a 104 kg','105 kg a 114 kg','115 kg e acima'}


## Core
>O core está separado em 4 laços **for**, sendo o primeiro para seleção do filtro de **homens/mulheres**, o segundo para seleção do filtro de **faixa etária**, o terceiro para navegação entre as **paginas**, e o quarto para extração dos **dados**.

1. O script clica no sexo masculino
2. Seleciona a primeira faixa etária (19 e abaixo)
3. Mantém selecionado o sexo masculino
4. Armazena os dados da primeira página na variável **rows**
5. Utiliza-se um laço **for** para extração dos dados e preenchimento das listas (citadas anteriormente)
6. Clica-se na próxima página - caso não tenha uma próxima, sai do atual laço de extração.
7. Seleciona-se a próxima faixa etária, e retorno para o passo 3

Durante os laços a **url_evento** é carregada algumas vezes para organizar a navegação do chrome.

>Após a extração dos dados cria-se 4 dataframes, e em seguida suas colunas são concatenadas num único dataframe, e é gerado um arquivo csv.

In [ ]:
for se in range(2,4):
    element=driver.find_element_by_xpath('//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/ul/li['+str(se)+']/a')
    driver.execute_script("arguments[0].click();", element) #clique do botão em javascript
    time.sleep(1)
    
    for fe in range(2,11):
        
        try:
            element=driver.find_element_by_xpath('//*[@id="premium-enhanced"]/ul/ul[1]/li['+str(fe)+']/a')
            driver.execute_script("arguments[0].click();", element) #clique do botão em javascript
            time.sleep(2)

            element=driver.find_element_by_xpath('//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/ul/li['+str(se)+']/a')
            driver.execute_script("arguments[0].click();", element) #clique do botão em javascript
            time.sleep(2)

            for page_number in range(0,ultima_pagina):
                dados = driver.find_element_by_id("segment-leaderboard")
                html=dados.get_attribute('innerHTML')
                soup = BeautifulSoup(html, "html.parser") #estruturação do código na variável soup
                table = soup.find("table", attrs={"class": "table table-striped table-padded table-leaderboard"}) #identificação da tabela
                rows = table.findAll("tr") # "tr" é o local no código onde estão as informações na tabela (linha)
                print('Página {p} de {u} do sexo {s} da faixa etária {f}'.format(p=page_number,u=ultima_pagina,s=se,f=fe))

                for row in rows:
                    a = [t.text.strip() for t in row.findAll("td")][0:] #"td" é um nível abaixo de "tr" (coluna para cada linha)
                    b = [c['href'] for c in row.find_all('a', href=True) if c.text] #identificação dos hyperlinks para a atividade do atleta
                    if len(a) > 0 and a != [''] and a !=['',''] and a != ['', '', ''] :
                        results.append(a)
                        result_faixa_etaria.append(faixa_etaria[str(fe)])
                        result_sexo.append(sexo[str(se)])
                        try:
                            link_atividades.append('https://strava.com' + b[1])
                        except Exception:
                            link_atividades.append(np.nan)


                try:
                    prox_pag=driver.find_element_by_link_text("→").click()


                except Exception:
                    driver.get(url_evento)
                    break
                time.sleep(2)

            driver.get(url_evento)
        except:
            next
data = []
for i, result in enumerate(results):
    data.append(results[i])

df_faixa_etaria=pd.DataFrame(result_faixa_etaria,columns=['faixa_etaria'])
df_sexo=pd.DataFrame(result_sexo,columns=['sexo'])
df_link_atividades=pd.DataFrame(link_atividades,columns=['segmento'])
df_link_atleta = pd.DataFrame(link_atleta,columns=['atleta'])


columns = ['classificacao','nome', 'ano','pace', 'fc','tempo_liquido']
df=pd.DataFrame(data,columns=columns)

df=pd.concat([df,df_faixa_etaria,df_sexo,df_link_atividades,df_link_atleta],axis=1)

df.drop(df[df['classificacao']=='Nenhum resultado encontrado'].index,inplace=True) #apagar as linhas que mostram como "nenhum arquivo encontrado", isso ocorre devido a faixas_etárias sem dados.

df.to_csv(exportbase,index=False)  
